<center><h2>ALTEGRAD Project </h2>
<h3>PREPROCESSING</h3>

<hr>
<span style="font-variant: small-caps;">Xavier Jiménez, Jean Quentin, Sacha Revol</span><br>
<hr>
</center>

# Imports

In [ ]:
import networkx as nx
import os
import csv
import numpy as np
import pandas as pd
from random import randint
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, make_scorer
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

# !pip install pip install karateclub
from gensim.models.doc2vec import Doc2Vec
from os import path
import pickle
from scipy import spatial
import random
from sklearn import preprocessing


# Preprocessing

Citation Graph is loaded from `edgelist.txt`

In [ ]:
# Create a graph
G = nx.read_edgelist('data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

Preprocess `authors.txt` and save it as a dictionary `authors_preprocessed.pkl`

In [ ]:
print('Preprocessing authors')
authors = dict()
with open('data/authors.txt', 'r', encoding="utf-8") as f:
    for line in tqdm(f):
        node, author = line.split('|--|')
        # author = author.lower()
        author = author.split(',')
        author[-1] = author[-1].strip()
        authors[int(node)] = author
    a_file = open("data/authors_preprocessed.pkl", "wb")
    pickle.dump(authors, a_file)
    a_file.close()
print('Preprocessing Done')
authors[0]

Preprocess `abstracts.txt` and save it as a dictionary `abstract_preprocessed.pkl`. Common NLP operations are made: lowercase, remove punctuation, tokenize, remove stop words.

In [ ]:
print('Preprocessing abstracts')
stop_words = stopwords.words('english')
porter = PorterStemmer()
abstracts = dict()
with open('data/abstracts.txt', 'r', encoding="utf-8") as f:
    for line in tqdm(f):
        node, abstract = line.split('|--|')
        abstract = abstract.lower()
        abstract = "".join([char for char in abstract if char not in string.punctuation])
        abstract = word_tokenize(abstract)
        abstract = [word for word in abstract if word not in stop_words]
        # abstract = [porter.stem(word) for word in abstract]
        abstracts[int(node)] = abstract
a_file = open("data/abstract_preprocessed.pkl", "wb")
pickle.dump(abstracts, a_file)
a_file.close()
print('Preprocessing Done')
abstracts[0][:10]

Citation Graph is loaded from `edgelist.txt` and random edges are removed from it (p=5%). These removed edges will be used to create a validation set that mirrors the test set `test.txt`. New graph is saved as `edgelist_val.txt` and will be loaded as H instead of G. If p is changed, the value should be changed as well on other .ipynb files.

In [ ]:
p = 0.05
    
H = nx.read_edgelist('data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)

with open('data/edgelist.txt') as file:
    lines = file.readlines()

random.seed(42)
indices_to_delete = random.sample(range(len(lines)), int(p * len(lines)))

# sort to delete biggest index first 
indices_to_delete.sort(reverse=True)

for i in tqdm(indices_to_delete):
    line = lines[i]
    t = line.split(',')
    H.remove_edge(int(t[0]), int(t[1]))

nx.readwrite.edgelist.write_edgelist(H, 'data/edgelist_val.txt', delimiter=',')